In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
batch_size = 32  # The default batch size of keras.
num_classes = 100  # Number of class for the dataset
epochs = 100

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

169001437/169001437 [==============================] - 3s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
def efficonvres(projection_dim):

    def apply(inputs):
        x_input = inputs

        x = layers.Conv2D(filters=projection_dim,kernel_size=3,padding="same",groups=projection_dim)(x_input)

        x1 = layers.DepthwiseConv2D(3, padding="same", use_bias=False)(x)
        x1 = layers.BatchNormalization(3)(x1)
        x1 = layers.Activation("swish")(x1)

        x1 = layers.DepthwiseConv2D(3, padding="same", use_bias=False)(x1)
        x1 = layers.LayerNormalization(epsilon=1e-6)(x1)
        x1 = layers.Activation("gelu")(x1)

        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x = layers.Dense(4 * projection_dim)(x)
        x = layers.Activation("gelu")(x)
        x = layers.Dense(projection_dim)(x)

        x = tf.keras.layers.Add()([x, x1])
        return x
    return apply

dims = [32, 64, 128, 256]
inputs = tf.keras.Input(shape=(32,32,3))
x = layers.Conv2D(filters=128,kernel_size=3,padding="same", activation='gelu')(inputs)
for i in range(4):
    x = layers.Conv2D(filters=dims[i],kernel_size=3,padding="same", activation='gelu')(x)
    x = efficonvres(dims[i])(x)
x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
output = layers.Dense(100, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=output)

In [5]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train, epochs=50, batch_size=64, 
                                  validation_data=(x_test, y_test))

Epoch 1/50
782/782 [==============================] - 198s 224ms/step - loss: 3.7824 - accuracy: 0.1241 - val_loss: 3.3541 - val_accuracy: 0.1821
Epoch 2/50
782/782 [==============================] - 172s 221ms/step - loss: 3.0759 - accuracy: 0.2412 - val_loss: 2.9996 - val_accuracy: 0.2430
Epoch 3/50
782/782 [==============================] - 172s 220ms/step - loss: 2.7275 - accuracy: 0.3060 - val_loss: 2.7461 - val_accuracy: 0.2983
Epoch 4/50
782/782 [==============================] - 172s 220ms/step - loss: 2.4928 - accuracy: 0.3544 - val_loss: 2.4998 - val_accuracy: 0.3527
Epoch 5/50
782/782 [==============================] - 172s 219ms/step - loss: 2.3191 - accuracy: 0.3914 - val_loss: 2.3706 - val_accuracy: 0.3718
Epoch 6/50
782/782 [==============================] - 171s 219ms/step - loss: 2.1654 - accuracy: 0.4230 - val_loss: 2.2452 - val_accuracy: 0.4050
Epoch 7/50
782/782 [==============================] - 171s 218ms/step - loss: 2.0471 - accuracy: 0.4518 - val_loss: 2.1527 -

In [7]:
model.save("efficonvres_100.h5")